<a href="https://colab.research.google.com/github/apoorva2810/CS6910/blob/main/Assignment-2/partA/partA_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Before executing the code, allocate GPU in Colab from: Edit -> Notebook Settings -> Hardware accelerator -> GPU

In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense,BatchNormalization
from keras.layers import Flatten
from keras.layers import Dropout
import random
import os
from google.colab import drive
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from tqdm import tqdm
from PIL import Image
import pickle
from keras.models import Model
from keras.layers import Input
from tensorflow.keras import layers, models, Model, optimizers
import tensorflow.keras as K
import cv2
import tensorflow.keras.backend as Kb
from keras import layers

from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint
from sklearn import preprocessing  
from keras.layers import LeakyReLU
from keras.models import Model
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing.image import img_to_array




In [ ]:
#To access content from google drive
drive.mount('/content/gdrive')

In [ ]:
#Todo- first rename the file in drive
#Use this link: https://drive.google.com/drive/folders/1yu5DuYlM_cvV5qwIKKnl3sHRG_G41aLB?usp=sharing
#To make copy of pickled data and save it folder name 'Raw_data350'

with open('/content/gdrive/My Drive/Raw_data350/xtrain','rb') as xT:
  xTrain = pickle.load(xT)

with open('/content/gdrive/My Drive/Raw_data350/ytrain','rb') as yT:
  yTrain = pickle.load(yT)

with open('/content/gdrive/My Drive/Raw_data350/xtest','rb') as xV:
  xTest = pickle.load(xV)

with open('/content/gdrive/My Drive/Raw_data350/ytest','rb') as yV:
  yTest = pickle.load(yV)

In [ ]:
xplot=[]
yplot=[]
x=10
for i in range(30):
  xplot.append(xTest[x])
  yplot.append(yTest[x])
  x=x+25


In [ ]:
len(yTest)

In [ ]:
xTrain, xVal, yTrain, yVal= train_test_split(xTrain, yTrain, test_size=.1)

In [ ]:
len(xTrain)

In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Train data

for i in range(len(xTrain)):

  xTrain[i]=np.array(xTrain[i])
  
  if(xTrain[i].ndim==2):
    del xTrain[i]
    del yTrain[i]

xTrain=np.array(xTrain)


In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Validation data

for i in range(len(xVal)):

  xVal[i]=np.array(xVal[i])
  
  if(xVal[i].ndim==2):
    del xVal[i]
    del yVal[i]

xVal=np.array(xVal)

In [ ]:
#Removing all the images which have only 2 dimensions and their respective y label from Train data

for i in range(len(xTest)):

  xTest[i]=np.array(xTest[i])
  
  if(xTest[i].ndim==2):
    del xTest[i]
    del yTest[i]

xTest=np.array(xTest)
type(xTrain)

In [ ]:
# Normalising xTrain
for i in range(len(xTrain)):
  # xTrain[i]= xTrain[i].astype('float32')
  xTrain[i]= xTrain[i]/255.0

In [ ]:
# Normalising xVal
for i in range(len(xVal)):

  xVal[i]=xVal[i].astype('float32')
  xVal[i]=xVal[i]/255.0

In [ ]:
# Normalising xTest
for i in range(len(xTest)):

  xTest[i]=xTest[i].astype('float32')
  xTest[i]=xTest[i]/255.0

In [ ]:
# Assigning the string labels of yTest and yTrain to integers
pple = preprocessing.LabelEncoder()
yTest=pple.fit_transform(yTest)
yVal=pple.fit_transform(yVal)
yTrain=pple.fit_transform(yTrain)

In [ ]:
#Change the integer y to one-hot vector
def to_categorical(y, n_class):
    return np.eye(n_class, dtype='uint8')[y]

In [ ]:
# getting the categorical value of yTrain and yTest using to_categorical() function
yTrain_cat=[]
for i in range(len(yTrain)):
  cat_d=to_categorical(int(yTrain[i]),10)
  yTrain_cat.append(cat_d)

yVal_cat=[]
for i in range(len(yVal)):
  cat_d=to_categorical(int(yVal[i]),10)
  yVal_cat.append(cat_d)

yTest_cat=[]
for i in range(len(yTest)):
  cat_d=to_categorical(int(yTest[i]),10)
  yTest_cat.append(cat_d)

In [ ]:
#changing the yTrain_cat and yTest_cat to numpy array 

yTrain_cat=np.array(yTrain_cat)
yVal_cat=np.array(yVal_cat)
yTest_cat=np.array(yTest_cat)

In [ ]:
print(yTrain_cat.shape)
print(yVal_cat.shape)
print(yTest_cat.shape)

In [ ]:
!pip install wandb

In [ ]:
import wandb
!wandb login

In [ ]:
sweep_config_temp={
  "name": "DL Assign-2-partA-final1",
  "method": "random",
  'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
  "parameters": {
        "num_filters": {
            "values": [64,128,256]
        },
        "filter_organisation":{
            "values":['same','half']
        },
        "activationFunction":{
            "values":['leakyRelu','Relu']  
        },
        "data_augumentation":{
            "values":['No']  
        },
        "filterSize":{
            "values": [(2,2),(3,3),(4,4)]  
        },
        "dropout":{
            "values":[0.2,0.3,0.5]
        },
        "batch_normalization":{
            "values":['Yes', 'No']
        },
    }
}

In [ ]:
sweep_id = wandb.sweep(sweep_config_temp, entity="cs20m014", project="DL Assign-2-partA-final")

In [ ]:
hyperparameter_defaults = dict(
    num_filters=64,
    filter_organisation='same',
    activationFunction='leakyRelu',
    data_augumentation='No',
    filterSize=(3,3),
    dropout=0.5,
    batch_normalization= 'Yes',
    )

In [ ]:
def model_run(eps, filter_size, mul_filter, drop, fSize, actv, batchNorm):

  current_filter_size=filter_size

  activationFun='relu'

  if actv=='leakyRelu':
    activationFun=LeakyReLU(alpha=0.1)

  model = Sequential()

  if batchNorm=='Yes':
    model.add(BatchNormalization(input_shape=(350, 350, 3)))

  

  model.add(Conv2D(current_filter_size, fSize, activation=activationFun, input_shape=(350, 350, 3)))
  model.add(MaxPooling2D((2, 2)))

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))

  if batchNorm=='Yes':
    model.add(BatchNormalization())

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
 
  model.add(MaxPooling2D((3, 3)))
  model.add(Flatten())
  model.add(Dropout(drop))


  model.add(Dense(1024, activation=activationFun))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  history = model.fit(xTrain,yTrain_cat,epochs = eps , validation_data = (xVal, yVal_cat))

  wandb.log({
        
        "Train Loss": history.history['loss'][eps-1],
        "Train Acc": history.history['accuracy'][eps-1],
        "Valid Loss":history.history['val_loss'][eps-1],
        "Valid Acc": history.history['val_accuracy'][eps-1]})
  return history.history['val_accuracy'][eps-1]
   

In [ ]:
def train():
  wandb.init(config=hyperparameter_defaults)
  config = wandb.config

  n_filters=config.num_filters
  multiplier=0

  if config.filter_organisation=='same':
    multiplier=1
  elif config.filter_organisation=='half':
    multiplier=0.5
  elif config.filter_organisation=='double':
    multiplier=2
  
  drop=config.dropout
  fSize=config.filterSize
  activation=config.activationFunction
  batchNorm = config.batch_normalization
  accuracy = model_run(5, n_filters, multiplier,drop,fSize, activation, batchNorm)
  

In [ ]:
wandb.agent(sweep_id, train)

**"""BEST MODEL"""**

part A 4a

In [ ]:
  eps=20
  filter_size= 128
  mul_filter=0.5 
  drop=.2
  fSize=(2,2)
  actv= 'leakyRelu'
  batchNorm='Yes'

  current_filter_size=filter_size

  activationFun='relu'

  if actv=='leakyRelu':
    activationFun=LeakyReLU(alpha=0.1)

  model = Sequential()

  if batchNorm=='Yes':
    model.add(BatchNormalization(input_shape=(350, 350, 3)))

  

  model.add(Conv2D(current_filter_size, fSize, activation=activationFun, input_shape=(350, 350, 3)))
  model.add(MaxPooling2D((2, 2)))

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))

  if batchNorm=='Yes':
    model.add(BatchNormalization())

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
 
  model.add(MaxPooling2D((3, 3)))
  model.add(Flatten())
  model.add(Dropout(drop))


  model.add(Dense(1024, activation=activationFun))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  history = model.fit(xTrain,yTrain_cat,epochs = eps , validation_data = (xTest, yTest_cat))


PART A 4C

In [ ]:
model1 = Model(inputs=model.inputs, outputs=model.layers[1].output)

In [ ]:
oneimg=xTest[60]

In [ ]:
oneimg = np.expand_dims(oneimg, axis=0)

In [ ]:
filter_vis = model1.predict(oneimg)

In [ ]:

filter_vis.shape

In [ ]:
#128 filters in best model , hence plotting as 16 x 8 grid


fig, axes = plt.subplots(nrows=16, ncols=8, figsize=(12, 20))
for idx,i in enumerate(axes.ravel()): 
    #print(i) visualising index by index
    imgl =filter_vis[0, :, :, idx-1]
    i.imshow(imgl, cmap = plt.get_cmap('gray'))
 
plt.show()

4B **PARTA** 

In [ ]:
xpredict=[]
k=10
for i in range(30):
  test_image = xTest[k]
  test_image = np.expand_dims(test_image, axis=0)
  prediction = model.predict(test_image)
  prediction = np.argmax(prediction[0])
  xpredict.append(prediction)
  k=k+25

In [ ]:
for i in range(len(xpredict)):
  if xpredict[i]==0:
    xpredict[i]='Amphibia'
  if xpredict[i]==1:
    xpredict[i]='Animalia'
  if xpredict[i]==2:
      xpredict[i]='Arachnida'
  if xpredict[i]==3:
      xpredict[i]='Aves'
  if xpredict[i]==4:
      xpredict[i]='Fungi'
  if xpredict[i]==5:
      xpredict[i]='Insecta'
  if xpredict[i]==6:
      xpredict[i]='Mammalia'
  if xpredict[i]==7 :
      xpredict[i]='Mollusca'
  if xpredict[i]==8 :
      xpredict[i]='Plantae'
  if xpredict[i]==9:
      xpredict[i]='Reptilia'
  

In [ ]:

i=0

fig, axes = plt.subplots(nrows=10, ncols=3, figsize=(12, 60))
for idx,ax in enumerate(axes.ravel()): 
    
    imgl = xplot[i]
    ax.imshow(imgl, cmap = plt.get_cmap('gray'))
    ax.title.set_text('Pred='+ xpredict[i]+'  ,  Act='+yplot[i])
    i=i+1
plt.show()

**best model **

In [ ]:
  eps=20
  filter_size= 128
  mul_filter=0.5 
  drop=.2
  fSize=(2,2)
  actv= 'leakyRelu'
  batchNorm='Yes'

  current_filter_size=filter_size

  activationFun='relu'

  if actv=='leakyRelu':
    activationFun=LeakyReLU(alpha=0.1)

  model = Sequential()

  if batchNorm=='Yes':
    model.add(BatchNormalization(input_shape=(350, 350, 3)))

  

  model.add(Conv2D(current_filter_size, fSize, activation=activationFun, input_shape=(350, 350, 3)))
  model.add(MaxPooling2D((2, 2)))

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))

  if batchNorm=='Yes':
    model.add(BatchNormalization())

  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
  
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(drop))
  if batchNorm=='Yes':
    model.add(BatchNormalization())
  
  current_filter_size=current_filter_size*mul_filter
  model.add(Conv2D(current_filter_size, fSize, activation=activationFun))
 
  model.add(MaxPooling2D((3, 3)))
  model.add(Flatten())
  model.add(Dropout(drop))


  model.add(Dense(1024, activation=activationFun))
  model.add(Dense(10, activation='softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  
  history = model.fit(xTrain,yTrain_cat,epochs = eps , validation_data = (xTest, yTest_cat))


## Question 5-Guided Backprop

In [ ]:
#Before running this code run the best model code in above cell

In [ ]:
img=xplot[9]

In [ ]:
img

In [ ]:
img = np.array(img)
x = np.expand_dims(img, axis=0)


In [ ]:
reluActv=tf.keras.activations.relu

In [ ]:
inputs = tf.cast(x, tf.float32)

In [ ]:
# inputs = np.float32(x)

In [ ]:
img.shape

In [ ]:
x.shape

In [ ]:
model.summary()

In [ ]:
inp=model.inputs
last_layer=model.get_layer('conv2d_19').output    #Choose the last convolution layer name from model.summary() for parameter in get_layer()

backProp_model = Model(
    inputs = [inp],    
    outputs = [last_layer]
)


In [ ]:
outputs = backProp_model(inputs)[0]


In [ ]:
actLayer=[]
for l in backProp_model.layers[1:]:
  if hasattr(l, 'activation'):
    actLayer.append(l)

In [ ]:
actLayer

In [ ]:
#  outputs = backProp_model(inputs)[0]

In [ ]:
#Defining function for guided relu and it's gradient
@tf.custom_gradient
def GR(x):
  def gradient(dy):
    return np.float32(x)  * np.float32(dy) *np.float32(dy) 
  return tf.nn.relu(x), gradient

In [ ]:
for l in actLayer:
    l.activation = GR

In [ ]:
backProp_model(inputs)[0].shape

In [ ]:
#Here we are actually calculating the gradient for back-propagation
with tf.GradientTape() as t:
  t.watch(inputs)   
  outputs = backProp_model(inputs)[0]
grads = t.gradient(outputs,inputs)[0]

In [ ]:
inputs.shape

In [ ]:
outputs.shape[2]

In [ ]:
outputs.shape

In [ ]:
grads.shape

In [ ]:
guided_back_prop[:,:,0].shape

In [ ]:
#Visualizing the guided back prop out
guided_back_prop =grads

R=guided_back_prop[:,:,0]
G=guided_back_prop[:,:,1]
B=guided_back_prop[:,:,2]

temp = np.dstack((R, G, B))       
temp = temp - np.min(temp)
temp = temp / temp.max()
    
backProp_image = plt.imshow(temp)
plt.show()

In [ ]:
print(backProp_image)